In [2]:
#carregando arquivo data.txt e criando RDD

import time
start = time.time()

from operator import add

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext.getOrCreate();

fileName = ('data.txt')
numPartitions = 4;
minSupport = 65000;

dados = sc.textFile(fileName);
arquivoRDD = sc.parallelize(dados.collect(), numPartitions) \
               .map(lambda x: x.split('\t'))

print(arquivoRDD.take(3));
print(type(arquivoRDD));

[['2,173,17,159,161,160,104,79,106,23,113,20,101,115,117,95,83,74,61,45,1124,1125,1202,9,1836,207,225,195,452,471,153,458,580,576,174,102,149,534,618,621,606,92,335,380,955,338,1079,895,336,573,826,565,141,339,1222,634,488,536,669,672,151,727,1019,1262,732,1080,1112,991,535,119,1113,1114,673,563,954,1081,752,751,1333,2191,1328,1319,1326,1332,7923,493,498,2717,792,729,861,2550,1153,1264,221'], ['3,17,159,161,160,32,104,79,95,83,20,74,26,102,162,108,92,25,23,124,132,97,44,27,15,188,166,489,880,796,965,966,1139,68,152,457,155,652,602,149,145,737,147,741,141,237,151,1234,35,1464,172,38,17,98,104,111,116,23,95,184,489,869,74,102,26,45,79,83,134,237,168,20,106,25,166,68,177,218,689,21,406,216,206,910,1070,896,952,990,1075,874,1130,799,620,492,115,138,762,980,600,197,692,579,769,809,631,151,596,679,1018,535,1019,144,395,818,1062,1222,678,409,638,746,550,1251,720,632,422,805,985,938,691,982,706,1256,887,1049,899,490,2319,2320,10,48,2674,2561,2387,2662,2576,937,2586,916,2663,636,2563,2373,2555,

In [3]:
#apenas mostra número de transações no RDD e um take em 3 elementos do RDD

numTransacoes = arquivoRDD.count();
print(numTransacoes); #numeros de transações
elementos = arquivoRDD.take(3)
print(elementos)
print(type(elementos));

136957
[['2,173,17,159,161,160,104,79,106,23,113,20,101,115,117,95,83,74,61,45,1124,1125,1202,9,1836,207,225,195,452,471,153,458,580,576,174,102,149,534,618,621,606,92,335,380,955,338,1079,895,336,573,826,565,141,339,1222,634,488,536,669,672,151,727,1019,1262,732,1080,1112,991,535,119,1113,1114,673,563,954,1081,752,751,1333,2191,1328,1319,1326,1332,7923,493,498,2717,792,729,861,2550,1153,1264,221'], ['3,17,159,161,160,32,104,79,95,83,20,74,26,102,162,108,92,25,23,124,132,97,44,27,15,188,166,489,880,796,965,966,1139,68,152,457,155,652,602,149,145,737,147,741,141,237,151,1234,35,1464,172,38,17,98,104,111,116,23,95,184,489,869,74,102,26,45,79,83,134,237,168,20,106,25,166,68,177,218,689,21,406,216,206,910,1070,896,952,990,1075,874,1130,799,620,492,115,138,762,980,600,197,692,579,769,809,631,151,596,679,1018,535,1019,144,395,818,1062,1222,678,409,638,746,550,1251,720,632,422,805,985,938,691,982,706,1256,887,1049,899,490,2319,2320,10,48,2674,2561,2387,2662,2576,937,2586,916,2663,636,2563,237

In [4]:
#cria tuplas de (ID do aluno, [ids das disciplinas])

tuplesRDD = arquivoRDD.map(lambda x : (x[0].split(",")[0],  x[0].split(",")[1:]));
print(tuplesRDD.take(2))

[('2', ['173', '17', '159', '161', '160', '104', '79', '106', '23', '113', '20', '101', '115', '117', '95', '83', '74', '61', '45', '1124', '1125', '1202', '9', '1836', '207', '225', '195', '452', '471', '153', '458', '580', '576', '174', '102', '149', '534', '618', '621', '606', '92', '335', '380', '955', '338', '1079', '895', '336', '573', '826', '565', '141', '339', '1222', '634', '488', '536', '669', '672', '151', '727', '1019', '1262', '732', '1080', '1112', '991', '535', '119', '1113', '1114', '673', '563', '954', '1081', '752', '751', '1333', '2191', '1328', '1319', '1326', '1332', '7923', '493', '498', '2717', '792', '729', '861', '2550', '1153', '1264', '221']), ('3', ['17', '159', '161', '160', '32', '104', '79', '95', '83', '20', '74', '26', '102', '162', '108', '92', '25', '23', '124', '132', '97', '44', '27', '15', '188', '166', '489', '880', '796', '965', '966', '1139', '68', '152', '457', '155', '652', '602', '149', '145', '737', '147', '741', '141', '237', '151', '1234'

In [5]:
#verticaliza a base e agrega tupla com código da disciplina e ids dos alunos (Código da Disciplina, [ids dos alunos])

vertical = tuplesRDD.flatMapValues(lambda x : x) 
verticalTuplesRDD = vertical.map(lambda x : (x[1], x[0])) #inverte disciplina e aluno
aggregateByKeyRDD = verticalTuplesRDD.groupByKey() \
                                     .map(lambda x : (x[0], list(x[1]))) \
                                     .sortByKey();
print(aggregateByKeyRDD.map(lambda x : (x[0], len(x[1]))).take(3))

[('10', 16620), ('100', 10788), ('1000', 2689)]


In [6]:
#aplica filtro de suporte mínimo nas disciplinas 

filterValuesRDD = aggregateByKeyRDD.filter(lambda x : len(x[1]) > minSupport) \
                                   .map(lambda x : ([x[0]],x[1]));

print(filterValuesRDD.count()); #count de disciplinas com mais do que o suporte mínimo
print(filterValuesRDD.map(lambda x : (x[0], len(x[1]))).take(5)); # (código da disciplina, [quantidade de alunos])


31
[(['129'], 144924), (['134'], 104148), (['135'], 147972), (['136'], 103728), (['138'], 100656)]


In [7]:
#funções

def intersect(a, b): return tuple(set(a) & set(b))
def unir(a,b) : return tuple(set(a + b))

In [8]:
#função usada na primeira geração de candidatos

def funcao(disc, rdd_collect):
    lista = [];
    for x in rdd_collect:
        if (not disc[0] == x[0]) & (disc[0] < x[0]):
            disciplinas = unir(disc[0],x[0])
            ids = intersect(disc[1],x[1])
            tupla = tuple(sorted(disciplinas)),tuple(sorted(ids))
            if not tupla in lista: lista += [tupla]
            #tupla = tuple(sorted(disciplinas)),(sorted(ids))
            #if not tupla in list: list += [tupla]
    return tuple(lista) 

In [9]:
#função usada nas outras gerações de candidatos

def funcao2(disc, rdd_collect):
    lista = []
    for x in rdd_collect:
        if disc[0] != x[0]:
            disciplinas = unir(disc[0],x[0])
            ids = intersect(disc[1],x[1])
            if (len(disciplinas)) == (len(disc[0]) + 1):
                tupla = tuple(sorted(disciplinas)),tuple(sorted(ids))
                #tupla = (sorted(disciplinas), sorted(ids))
                if not(tupla in lista):lista += [tupla]
    return tuple(lista)

In [10]:
# map com funçao para primeira geração de candidatos + filter de suporte mínimo

disciplinas_singleton = filterValuesRDD.collect()
filterDisciplinasRDD = filterValuesRDD.flatMap(lambda x : funcao(x, disciplinas_singleton)) \
                                      .filter(lambda x : len(x[1]) > minSupport) \
                                      .sortByKey()

contagem = filterDisciplinasRDD.map(lambda x : (x[0], len(x[1])))
print(contagem.collect())

[(('129', '135'), 70992), (('129', '139'), 69900), (('129', '153'), 70512), (('129', '17'), 66984), (('135', '139'), 70092), (('135', '153'), 70260), (('135', '17'), 66288), (('139', '153'), 70032), (('139', '17'), 65916), (('147', '17'), 65736), (('153', '17'), 67080)]


In [11]:
# maps com função para geração de próximos candidatos + reduce + filter de suporte mínimo

disciplinas2itemsets = filterDisciplinasRDD.collect()

filterDisciplinasRDD2 = filterDisciplinasRDD.flatMap(lambda x : funcao2(x, disciplinas2itemsets)) \
                                            .map(lambda x : (x,1)) \
                                            .reduceByKey(add) \
                                            .sortByKey() \
                                            .map(lambda x : x[0]) \
                                            .filter(lambda x : len(x[1]) > minSupport)
    
contagem2 = filterDisciplinasRDD2.map(lambda x : (x[0], len(x[1])))
print(contagem2.collect())

[(('129', '135', '139'), 69444), (('129', '135', '153'), 69684), (('129', '135', '17'), 65400), (('129', '139', '153'), 69288), (('129', '139', '17'), 65100), (('129', '153', '17'), 65532), (('135', '139', '153'), 69372), (('135', '139', '17'), 65124), (('135', '153', '17'), 65160), (('139', '153', '17'), 65292)]


In [12]:
disciplinas3itemsets = filterDisciplinasRDD2.collect()

filterDisciplinasRDD3 = filterDisciplinasRDD2.flatMap(lambda x : funcao2(x, disciplinas3itemsets)) \
                                             .map(lambda x : (x,1)) \
                                             .reduceByKey(add) \
                                             .sortByKey() \
                                             .map(lambda x : x[0]) \
                                             .filter(lambda x : len(x[1]) > minSupport)
    
contagem3 = filterDisciplinasRDD3.map(lambda x : (x[0], len(x[1])))
print(contagem3.collect())

[(('129', '135', '139', '153'), 68976)]


In [13]:
disciplinas4itemsets = filterDisciplinasRDD3.collect()

filterDisciplinasRDD4 = filterDisciplinasRDD3.flatMap(lambda x : funcao2(x, disciplinas4itemsets)) \
                                             .map(lambda x : (x,1)) \
                                             .reduceByKey(add) \
                                             .sortByKey() \
                                             .map(lambda x : x[0]) \
                                             .filter(lambda x : len(x[1]) > minSupport)
    
contagem4 = filterDisciplinasRDD4.map(lambda x : (x[0], len(x[1])))
print(contagem4.collect())

[]


In [14]:
# Tempo de execução:
end = time.time()
tempo = (end - start)
print("Com ",numPartitions," partições | Tempo de Execução: ", tempo)

Com  4  partições | Tempo de Execução:  342.8978066444397
